In [1]:
# вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.value = None


class Trie:
    def __init__(self):
        self.root = TrieNode()
        self.size = 0

    def put(self, key, value=None):
        if not isinstance(key, str) or not key:
            raise TypeError(
                f"Illegal argument for put: key = {key} must be a non-empty string"
            )

        current = self.root
        for char in key:
            if char not in current.children:
                current.children[char] = TrieNode()
            current = current.children[char]
        if current.value is None:
            self.size += 1
        current.value = value

    def get(self, key):
        if not isinstance(key, str) or not key:
            raise TypeError(
                f"Illegal argument for get: key = {key} must be a non-empty string"
            )

        current = self.root
        for char in key:
            if char not in current.children:
                return None
            current = current.children[char]
        return current.value

    def delete(self, key):
        if not isinstance(key, str) or not key:
            raise TypeError(
                f"Illegal argument for delete: key = {key} must be a non-empty string"
            )

        def _delete(node, key, depth):
            if depth == len(key):
                if node.value is not None:
                    node.value = None
                    self.size -= 1
                    return len(node.children) == 0
                return False

            char = key[depth]
            if char in node.children:
                should_delete = _delete(node.children[char], key, depth + 1)
                if should_delete:
                    del node.children[char]
                    return len(node.children) == 0 and node.value is None
            return False

        return _delete(self.root, key, 0)

    def is_empty(self):
        return self.size == 0

    def longest_prefix_of(self, s):
        if not isinstance(s, str) or not s:
            raise TypeError(
                f"Illegal argument for longestPrefixOf: s = {s} must be a non-empty string"
            )

        current = self.root
        longest_prefix = ""
        current_prefix = ""
        for char in s:
            if char in current.children:
                current = current.children[char]
                current_prefix += char
                if current.value is not None:
                    longest_prefix = current_prefix
            else:
                break
        return longest_prefix

    def keys_with_prefix(self, prefix):
        if not isinstance(prefix, str):
            raise TypeError(
                f"Illegal argument for keysWithPrefix: prefix = {prefix} must be a string"
            )

        current = self.root
        for char in prefix:
            if char not in current.children:
                return []
            current = current.children[char]

        result = []
        self._collect(current, list(prefix), result)
        return result

    def _collect(self, node, path, result):
        if node.value is not None:
            result.append("".join(path))
        for char, next_node in node.children.items():
            path.append(char)
            self._collect(next_node, path, result)
            path.pop()

    def keys(self):
        result = []
        self._collect(self.root, [], result)
        return result

In [3]:
class LongestCommonWord(Trie):

    def find_longest_common_word(self, strings) -> str:
        """
        Знаходить найдовше спільний префікс в списку рядків.
        :param strings: список рядків
        :return: найдовше спільне слово

        Головная ідея полягає в використанні тієї властивості Trie(), що це дерево.
        Тож ми будуємо тимчасовий Trie з усіх рядків, а потім йдемо по дереву вниз від кореня,
        поки не знайдемо вузол з більш ніж одним нащадком (розгалудження дерева), що вказує на те,
        що ми вже знайшли спільний префікс.

        Складність алгоритму O(n), де N - загальна довжина всіх рядків. При цьому O(n) - це складність створення дерева, 
        а складність знаходження спільного префікса - O(m), де m - довжина спільного префікса. 
        За рахунок використання Trie() складність пошуку така невелика, що говорить про ефективність такої структури даних.

        Треба завначити, що тут в завданні дивний момент - ми пишемо метод об'єкту, але не використовуємо self.
        Логічніше було б написати його як статичний метод або як окрему функцію, де Trie() буде просто інструментом для оптимізації алгоритму,
        але я зробив саме так, як сказано в умовах ззавдання.
        """
        if not isinstance(strings, list) or not all(isinstance(s, str) for s in strings):
            raise TypeError(
                f"Illegal argument for find_longest_common_word: strings = {strings} must be a list of strings"
            )

        temp_trie = Trie()
        for string in strings:
            temp_trie.put(string)

        longest_common_prefix = ""
        current_node = temp_trie.root
        while current_node.children:
            if len(current_node.children) > 1:
                break
            char = list(current_node.children.keys())[0]
            longest_common_prefix += char
            current_node = current_node.children[char]

        return longest_common_prefix


if __name__ == "__main__":
    # Тести
    trie = LongestCommonWord()
    strings = ["flower", "flow", "flight"]
    assert trie.find_longest_common_word(strings) == "fl"

    trie = LongestCommonWord()
    strings = ["interspecies", "interstellar", "interstate"]
    assert trie.find_longest_common_word(strings) == "inters"

    trie = LongestCommonWord()
    strings = ["dog", "racecar", "car"]
    assert trie.find_longest_common_word(strings) == ""